<a href="https://colab.research.google.com/github/gabi-mendes/PyError/blob/main/PyError.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Estrutura de Dados para Propagação de Erros 

Para a confecção do programa, usaremos apenas um módulo da biblioteca padrão do Python, o módulo math.


In [1]:
import math

A estrutura de dados será nomeada de DataPoint, que terá uma variável para o seu valor e outra para o seu erro. Utilizando as funções \_\_add__, \_\_sub__, \_\_mul__ e \_\_truediv__ podemos determinar como se dará a soma, subtração, multiplicação e divisão dessa classe, respectivamente. Além disso, a função error_propagation terá um papel muito importante para o cálculo do erro dessas operações.

In [2]:
class DataPoint:
    ADD = "+"
    SUB = "-"
    MUL = "*"
    DIV = "/"

    def __init__(self, value, error=.0):
        self.value = value
        self.error = error

    def __add__(self, other):
        if isinstance(other, float) or isinstance(other, int):
            self.value += other
            return self

        if isinstance(other, DataPoint):
            return DataPoint(self.value + other.value,
                             self.error_propagation(self.ADD, other))

        raise TypeError

    def __sub__(self, other):
        if isinstance(other, float) or isinstance(other, int):
            self.value -= other
            return self

        if isinstance(other, DataPoint):
            return DataPoint(self.value - other.value,
                             self.error_propagation(self.SUB, other))

        raise TypeError

    def __mul__(self, other):
        if isinstance(other, float) or isinstance(other, int):
            self.error = self.error_propagation(self.MUL, other)
            self.value = self.value * other
            return self

        if isinstance(other, DataPoint):
            return DataPoint(self.value * other.value,
                             self.error_propagation(self.MUL, other))

        raise TypeError

    def __truediv__(self, other):
        if isinstance(other, float) or isinstance(other, int):
            self.error = self.error_propagation(self.DIV, other)
            self.value = self.value / other
            return self

        if isinstance(other, DataPoint):
            return DataPoint(self.value / other.value,
                             self.error_propagation(self.DIV, other))

        raise TypeError
    
    @property
    def max_value(self):
        return self.value + self.error
        
    @property
    def min_value(self):
        return self.value - self.error

    @staticmethod
    def combination(data_points):
        n = len(data_points)

        sigma = .0
        for index in range(n):
            if not isinstance(data_points[index], DataPoint):
                raise TypeError
            
            sigma += 1 / data_points[index].error ** 2

        sigma = 1 / math.sqrt(sigma)

        value = .0
        for index in range(n):
            value += ((sigma / data_points[index].error) ** 2) * data_points[index].value

        return DataPoint(value, sigma)

    @staticmethod
    def compatibility(data_points):
        n = len(data_points)

        for i in range(n):
            for j in range(n):
                if i == j:
                    continue
                
                dp = data_points[i] - data_points[j]

                if abs(dp.value) > 2 * dp.error:
                    return False

        return True
        


    def error_propagation(self, operator, other):
        if operator == self.ADD or operator == self.SUB:
            if isinstance(other, DataPoint):
                return math.sqrt(self.error ** 2 + other.error ** 2)

            if isinstance(other, (float, int)):
                return self.error

            raise TypeError

        if operator == self.MUL:
            if isinstance(other, DataPoint):
                return (self.value * other.value) * math.sqrt(
                    (self.error / self.value) ** 2 + (other.error / other.value) ** 2)

            if isinstance(other, (float, int)):
                return other * self.error

            raise TypeError

        if operator == self.DIV:
            if isinstance(other, DataPoint):
                return (self.value / other.value) * math.sqrt(
                    (self.error / self.value) ** 2 + (other.error / other.value) ** 2)
            
            if isinstance(other, (float, int)):
                return self.error / other

            raise TypeError

Fazendo um teste,

In [3]:
data1 = DataPoint(10., 1.)
data2 = DataPoint(25., 1.4)
data3 = data1 * data2

print(f"Valor: {data3.value} / Erro: {data3.error}")

Valor: 250.0 / Erro: 28.653097563788805


Para uma quantidade N de dados, podemos criar uma DataList.

In [4]:
def data_list(points):
    data_points = []
    for point in points:
        if isinstance(point, list):
            data_points.append(DataPoint(point[0], point[1]))

        else:
            data_points.append(DataPoint(point))

    return data_points


# Experimento de Carga-Massa

Para o experimento de carga-massa, temos os seguintes dados:

In [5]:
v = data_list([175, 200, 225, 250, 275])
i = data_list([1.25, 1.50, 1.75, 2.00, 1.50])
r = data_list([[4.70, 0.5], [4.20, 0.5], [3.85, 0.5], [3.50, 0.5], [4.90, 0.5]])

r_si = []
for value in r:    
    r_si.append(value / 100)

Realizando os cálculos,

In [6]:
k = DataPoint(125 * (0.149) ** 2 / (32 * 124 ** 2 * 1.26e-6 ** 2))

def charge_mass(V, I, R):
    return k * V / ((I * R) * (I * R)) 

charge_mass_list = []
for j in range(len(v)):
    charge_mass_value = charge_mass(v[j], i[j], r[j])
    print(f"{charge_mass_value.value} +- {charge_mass_value.error}")
    charge_mass_list.append(charge_mass_value)

180123516819.76398 +- 27099268126.473244
179018126486.90686 +- 30139269331.95308
176089342835.49347 +- 32341290497.05461
181255853067.99316 +- 36619212238.32298
180844842063.3038 +- 26097268197.055576


Realizando a compatibilidade e a combinação.

In [7]:
if DataPoint.compatibility(charge_mass_list):
    final_value = DataPoint.combination(charge_mass_list)
    print(f"{final_value.value} +- {final_value.error}")

179560678352.19757 +- 13324274116.400427
